In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import cluster
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv(r"data.csv")

In [3]:
data.rename(columns = {'Do you prefer a gas or electric stovetop?' : 'gas/electric',
                    'For how long are you looking to rent?' : 'rent length',
                    'How many rooms do you want?':'rooms',
                    'What is your budget for rent?':'budget',
                    'How many bathrooms do you want?':'bathrooms',
                    'How important is it that you have a master bedroom?':'master',
                    'How important is it that you have a storage unit?':'storage',
                    'How important is it that you a balcony/garden?':'balcony/garden',
                    'How important is it that you have also an electric heater (דוד חשמל) and a regular heater (דוד שמש)?':'electric/solar',
                    'How important, is it that you have an air-conditioner in every bedroom? ':'AC',
                    'How important is it that you have personal parking spot(s)?':'parking',
                    'How Important is it that you have a kosher Kitchen?':'SK',
                    'How important is it that you have bus routes near you?':'public transpo',
                    'Does it matter if the apartment is next to a school or a apartment complex under construction?':'loud neighborhood',
                    'Does it matter to you to be in a building that can undergo construction in the coming year/two years?':'construction to building',
                    'How Important is it that your apartment come furnished?':'furnished',
                    'How important that the apartment be pet friendly?':'pets',
                    'How important that the apartment be smoke friendly?':'smoke',
                    'How important that the apartment have a big livingroom?':'livingroom',
                    'On which Floor would you want the apartment to be on?':'floor',
                    'How important is it which floor you are on?':'floor num',
                    'How important that the apartment have elevator access?':'accesiable',
                    'Where are you looking for an apartment?':'where',}
          , inplace = True)
data.head()

data.drop('Timestamp', inplace=True, axis=1)
data.drop('Would you rather your neighbors be students or families?', inplace=True, axis=1)
data.drop('Is it Important that your neighbors be families?', inplace=True, axis=1)
data.drop('Do you prefer your neighbors be families or singles?', inplace=True, axis=1)
data.drop('Is it important that your neighbors be students', inplace=True, axis=1)
data.drop('What type of neighborhood are you looking for?', inplace=True, axis=1)



In [4]:
data["gas/electric"].fillna(3, inplace = True)
data["bathrooms"].fillna(1.5, inplace = True)
data["master"].fillna(3, inplace = True)
data["storage"].fillna(3, inplace = True)
data["balcony/garden"].fillna(3, inplace = True)
data["electric/solar"].fillna(3, inplace = True)
data["AC"].fillna(3, inplace = True)
data["parking"].fillna(3, inplace = True)
data["SK"].fillna(3, inplace = True)
data["public transpo"].fillna(3, inplace = True)
data["loud neighborhood"].fillna(3, inplace = True)
data["construction to building"].fillna(3, inplace = True)
data["furnished"].fillna(3, inplace = True)
data["smoke"].fillna(3, inplace = True)
data["pets"].fillna(3, inplace = True)
data["livingroom"].fillna(3, inplace = True)
data["accesiable"].fillna(3, inplace = True)
data["rent length"].replace("1 Month", 1, regex=True, inplace = True)
data["rent length"].replace("Half a Year", 6, regex=True, inplace = True)
data["rent length"].replace("A Year", 12, regex=True, inplace = True)
data["rent length"].replace("2 years", 24, regex=True, inplace = True)
data["rent length"].replace("2 Months", 2, regex=True, inplace = True)
data["budget"].replace("6100-7000, 7100-8000", 8000, regex=True, inplace = True)
data["budget"].replace("6100-7000", 7000, regex=True, inplace = True)
data["budget"].replace("5100-6000", 6000, regex=True, inplace = True)
data["budget"].replace("4100-5000, 5100-6000", 6000, regex=True, inplace = True)
data["budget"].replace("3100-4000", 4000, regex=True, inplace = True)
data["budget"].replace("2100-3000", 3000, regex=True, inplace = True)
data["budget"].replace("1000-2000", 2000, regex=True, inplace = True)

In [5]:
phonenumber=[]
name=[]
for i in range(len(data)):
    phonenumber.append(1000000+i)
    name.append("person"+ str(i))
data["phone number"]= phonenumber
data["name"]=name

for i in range(data.shape[0]):
    data["rooms"].replace(data["rooms"][i], data["rooms"][i][-1], regex=True, inplace = True)
data["rooms"] = data["rooms"].astype(int)


In [6]:
data.head()

,gas/electric,rent length,rooms,budget,bathrooms,master,storage,balcony/garden,electric/solar,AC,...,furnished,pets,smoke,livingroom,floor,floor num,accesiable,where,phone number,name
0,3,6,4,3000,2.0,3,3,1,3.0,5,...,5,1.0,1,3,"First, Second",1,1,"City, Jerusalem",1000000,person0
1,3,12,4,3000,1.0,3,2,3,1.0,3,...,3,1.0,1,2,"Ground, First, Second, Third, Fourth",2,2,בקעה ירושלים,1000001,person1
2,3,12,6,8000,2.0,2,2,5,3.0,5,...,4,5.0,4,4,"Ground, First, Second",1,4,City,1000002,person2
3,4,12,3,4000,1.0,3,3,3,5.0,2,...,3,1.0,1,4,"Second, Third",4,3,City,1000003,person3
4,1,24,3,4000,1.0,5,3,5,3.0,1,...,3,5.0,5,4,"Ground, First",1,4,Yeshuv,1000004,person4


In [7]:
# data.to_csv(r'C:\Users\talia\Hackathon\Hachathon_May22\data_updated.csv')

### Algo for finding roomates

we will be getting a json with the column names in the same order as the df, some of the columns may be empty
iterate threw the rows of the df </br>
    compare each value in the df to the json value, if equal or better give 1.
    for some values if non comparable change to 0 that is score, for others give 0 for that column
    
    flag is counter to how many things totally dont match nut are an ultimatum
    
#### ultamatum (pet,smoke)
if abs(diff>=3) total 0 end this row
dif(2)=0.5
dif(<=1)=1

#### non ultametium(floor num)
if very different give zero flag to divide score by 2

#### 1-5 rank how important
find diff between json and column if abs(diff==0,1)=1 else 0.5
   
  
#### 1-5 rank opossites
find diff between json and column if abs(diff==1)=0.75 if abs(diff=2)=0.5 if abs(diff=3)=0.25 if abs(diff=4)=0 if abs(diff=0) score=1
  
    
    
### Algo for finding apartments
we will be getting a json with the column names in the same order as the df, some of the columns may be empty
iterate threw the rows of the df </br>
    compare each value in the df to the json value, if equal or better give 1.
    for some values if non comparable change to 0 that is score, for others give 0 for that column
    
    
    
#### ultamatum (pet,smoke)
if abs(diff>=3) total 0 end this row
dif(2)=0.5
dif(<=1)=1    


#### non ultametium(floor num, parking, garden/balcany)
if very different give zero flag to divide score by 2


#### 1-5 rank opossites
find diff between json and column if abs(diff==1)=0.75 if abs(diff=2)=0.5 if abs(diff=3)=0.25 if abs(diff=4)=0 if abs(diff=0) score=1

In [8]:
building=pd.read_csv("Airbnb_Milan.csv")

In [9]:
building.head()

,gas/electric,livingroom,bedrooms,bathrooms,city?,where,languages,house/building,storage?,furnished?,...,accessible?,Elevator,parking,pets,smoke,price,length of rent,public transpo,near construction,loud neighborhood
0,1,1,2,1.0,1,Ariel,EN,0,1,1,...,0,0,1,0,0,3000,1,1,0,1
1,0,1,2,1.0,1,Ariel,AM,1,0,1,...,1,1,0,0,0,3000,1,1,0,1
2,1,1,2,1.0,1,Ariel,FR,1,1,1,...,1,1,0,0,0,3000,1,1,0,1
3,0,1,1,1.0,1,Haifa,EN,1,1,1,...,0,1,1,1,0,2000,1,1,0,1
4,0,1,1,1.5,1,Ariel,AM,1,1,1,...,1,1,0,0,0,2200,1,1,0,1


In [10]:
phonenumber=[]
name=[]
for i in range(len(building)):
    phonenumber.append(110000+i)
    name.append("owner"+str(i))
building["phone number"]= phonenumber
building["name"]=name

In [11]:
building.head()

,gas/electric,livingroom,bedrooms,bathrooms,city?,where,languages,house/building,storage?,furnished?,...,parking,pets,smoke,price,length of rent,public transpo,near construction,loud neighborhood,phone number,name
0,1,1,2,1.0,1,Ariel,EN,0,1,1,...,1,0,0,3000,1,1,0,1,110000,owner0
1,0,1,2,1.0,1,Ariel,AM,1,0,1,...,0,0,0,3000,1,1,0,1,110001,owner1
2,1,1,2,1.0,1,Ariel,FR,1,1,1,...,0,0,0,3000,1,1,0,1,110002,owner2
3,0,1,1,1.0,1,Haifa,EN,1,1,1,...,1,1,0,2000,1,1,0,1,110003,owner3
4,0,1,1,1.5,1,Ariel,AM,1,1,1,...,0,0,0,2200,1,1,0,1,110004,owner4


In [12]:
# building.to_csv(r'C:\Users\talia\Hackathon\Hachathon_May22\building_updated.csv')

# clustering

# FULL DATA

In [13]:
data = pd.read_csv(r'data_updated.csv')
data.drop("Unnamed: 0", inplace=True, axis=1)
data

,gas/electric,rent length,bedrooms,price,bathrooms,master,storage?,balcony/garden,electric/solar,AC,...,construction to building,furnished?,pets,smoke,livingroom,accessible?,where,city,phone number,name
0,3,6.0,4,3000.0,2.0,3,3,1,3,5,...,3,5,1,1,3,1,Jerusalem,1.0,1000000,person0
1,3,12.0,4,3000.0,1.0,3,2,3,1,3,...,3,3,1,1,2,2,Jerusalem,1.0,1000001,person1
2,3,12.0,6,8000.0,2.0,2,2,5,3,5,...,3,4,5,4,4,4,Beer Sheba,1.0,1000002,person2
3,4,12.0,3,4000.0,1.0,3,3,3,5,2,...,4,3,1,1,4,3,Beer Sheba,1.0,1000003,person3
4,1,24.0,3,4000.0,1.0,5,3,5,3,1,...,5,3,5,5,4,4,,0.0,1000004,person4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,4,12.0,3,3000.0,1.0,1,1,5,4,5,...,5,3,1,4,4,2,Merkaz,1.0,1001193,person1193
1194,5,12.0,2,3000.0,1.0,4,1,5,5,5,...,5,5,5,5,3,3,,1.0,1001194,person1194
1195,5,12.0,4,2000.0,2.0,3,1,5,4,5,...,1,5,1,3,5,3,,1.0,1001195,person1195
1196,4,12.0,3,4000.0,1.0,1,1,4,2,3,...,5,1,1,1,4,1,Jerusalem,1.0,1001196,person1196


In [14]:
building = pd.read_csv(r'building_updated.csv')
building.drop("Unnamed: 0", inplace=True, axis=1)
building

,gas/electric,livingroom,bedrooms,bathrooms,city,where,languages,house/building,storage?,furnished?,...,parking,pets,smoke,price,rent length,public transpo,near construction,loud neighborhood,phone number,name
0,1,1,2,1.0,1,Ariel,EN,0,1,1,...,1,0,0,3000,1,1,0,1,110000,owner0
1,0,1,2,1.0,1,Ariel,AM,1,0,1,...,0,0,0,3000,1,1,0,1,110001,owner1
2,1,1,2,1.0,1,Ariel,FR,1,1,1,...,0,0,0,3000,1,1,0,1,110002,owner2
3,0,1,1,1.0,1,Haifa,EN,1,1,1,...,1,1,0,2000,1,1,0,1,110003,owner3
4,0,1,1,1.5,1,Ariel,AM,1,1,1,...,0,0,0,2200,1,1,0,1,110004,owner4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9317,1,0,1,1.0,1,Ariel,0,1,1,1,...,0,0,0,2000,12,1,0,0,119317,owner9317
9318,0,1,2,1.0,1,Ariel,EN,1,0,1,...,0,1,0,3000,12,1,0,0,119318,owner9318
9319,0,0,1,1.0,1,Ariel,0,1,1,1,...,0,0,0,1500,12,1,0,0,119319,owner9319
9320,0,0,1,1.0,1,Ariel,0,1,1,1,...,0,0,0,1500,12,1,0,0,119320,owner9320


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gas/electric              1198 non-null   int64  
 1   rent length               1190 non-null   float64
 2   bedrooms                  1198 non-null   int64  
 3   price                     1186 non-null   float64
 4   bathrooms                 1198 non-null   float64
 5   master                    1198 non-null   int64  
 6   storage?                  1198 non-null   int64  
 7   balcony/garden            1198 non-null   int64  
 8   electric/solar            1198 non-null   int64  
 9   AC                        1198 non-null   int64  
 10  parking                   1198 non-null   int64  
 11  SK                        1198 non-null   int64  
 12  public transpo            1198 non-null   int64  
 13  loud neighborhood         1198 non-null   int64  
 14  construc

In [16]:
# where: {'Jerusalem ': 0, 'Beer Sheba': 1, ' ': 2, 'Merkaz': 3, 'Haifa': 4, 'Otef Azza': 5, 'North': 6, 'Tel Aviv': 7, 'Ariel': 8, 'South': 9, 'Tel aviv': 10}
u = data['where'].unique()
u_int = {}
for i in range(len(u)):
    u_int[u[i]] = i
w = []    
for index, row in data.iterrows():
    row['where'] = u_int.get(row['where'])
    w.append(row['where'])
data['where'] = w
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gas/electric              1198 non-null   int64  
 1   rent length               1190 non-null   float64
 2   bedrooms                  1198 non-null   int64  
 3   price                     1186 non-null   float64
 4   bathrooms                 1198 non-null   float64
 5   master                    1198 non-null   int64  
 6   storage?                  1198 non-null   int64  
 7   balcony/garden            1198 non-null   int64  
 8   electric/solar            1198 non-null   int64  
 9   AC                        1198 non-null   int64  
 10  parking                   1198 non-null   int64  
 11  SK                        1198 non-null   int64  
 12  public transpo            1198 non-null   int64  
 13  loud neighborhood         1198 non-null   int64  
 14  construc

In [17]:
data = data.dropna(axis=0, how='any')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1196
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gas/electric              1028 non-null   int64  
 1   rent length               1028 non-null   float64
 2   bedrooms                  1028 non-null   int64  
 3   price                     1028 non-null   float64
 4   bathrooms                 1028 non-null   float64
 5   master                    1028 non-null   int64  
 6   storage?                  1028 non-null   int64  
 7   balcony/garden            1028 non-null   int64  
 8   electric/solar            1028 non-null   int64  
 9   AC                        1028 non-null   int64  
 10  parking                   1028 non-null   int64  
 11  SK                        1028 non-null   int64  
 12  public transpo            1028 non-null   int64  
 13  loud neighborhood         1028 non-null   int64  
 14  construc

In [19]:
personal_info = data[['phone number', 'name']]

data.drop('phone number', inplace=True, axis=1)
data.drop('name', inplace=True, axis=1)
data

,gas/electric,rent length,bedrooms,price,bathrooms,master,storage?,balcony/garden,electric/solar,AC,...,public transpo,loud neighborhood,construction to building,furnished?,pets,smoke,livingroom,accessible?,where,city
0,3,6.0,4,3000.0,2.0,3,3,1,3,5,...,5,1,3,5,1,1,3,1,0,1.0
1,3,12.0,4,3000.0,1.0,3,2,3,1,3,...,4,4,3,3,1,1,2,2,0,1.0
2,3,12.0,6,8000.0,2.0,2,2,5,3,5,...,5,4,3,4,5,4,4,4,1,1.0
3,4,12.0,3,4000.0,1.0,3,3,3,5,2,...,4,2,4,3,1,1,4,3,1,1.0
4,1,24.0,3,4000.0,1.0,5,3,5,3,1,...,5,4,5,3,5,5,4,4,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,5,12.0,5,2000.0,2.0,1,3,3,5,4,...,5,3,3,4,3,1,3,4,2,1.0
1193,4,12.0,3,3000.0,1.0,1,1,5,4,5,...,1,3,5,3,1,4,4,2,3,1.0
1194,5,12.0,2,3000.0,1.0,4,1,5,5,5,...,2,5,5,5,5,5,3,3,2,1.0
1195,5,12.0,4,2000.0,2.0,3,1,5,4,5,...,4,1,1,5,1,3,5,3,2,1.0


In [20]:
# Fitting K-Means to the dataset
kmeans_data = KMeans(n_clusters = 100, init = 'k-means++', random_state = 42)
y_kmeans_data = kmeans_data.fit_predict(data)
y_kmeans_data

array([94, 76, 10, ..., 39, 69,  2])

In [21]:
# # Visualising the clusters
# plt.figure(figsize=(15,7))
# t = data.values
# sns.scatterplot(t[y_kmeans == 0, 0], t[y_kmeans == 0, 1], color = 'yellow', label = 'Cluster 1',s=50)
# sns.scatterplot(t[y_kmeans == 1, 0], t[y_kmeans == 1, 1], color = 'blue', label = 'Cluster 2',s=50)
# sns.scatterplot(t[y_kmeans == 2, 0], t[y_kmeans == 2, 1], color = 'green', label = 'Cluster 3',s=50)
# sns.scatterplot(t[y_kmeans == 3, 0], t[y_kmeans == 3, 1], color = 'grey', label = 'Cluster 4',s=50)
# sns.scatterplot(t[y_kmeans == 4, 0], t[y_kmeans == 4, 1], color = 'orange', label = 'Cluster 5',s=50)
# sns.scatterplot(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color = 'red', 
#                 label = 'Centroids',s=300,marker=',')
# plt.grid(False)
# plt.title('Clusters of customers')
# plt.xlabel('Annual Income (k$)')
# plt.ylabel('Spending Score (1-100)')
# plt.legend()
# plt.show()

In [22]:
# cl_data = cluster.FeatureAgglomeration(n_clusters = 10, affinity = 'euclidean', linkage = 'ward').fit(data.T)
# AgglomerativeClustering()
# cl_data.labels_

In [23]:
personal_info['cluster'] = kmeans_data.labels_
personal_info

,phone number,name,cluster
0,1000000,person0,94
1,1000001,person1,76
2,1000002,person2,10
3,1000003,person3,23
4,1000004,person4,41
...,...,...,...
1192,1001192,person1192,52
1193,1001193,person1193,17
1194,1001194,person1194,39
1195,1001195,person1195,69


In [24]:
full_data = data.join(personal_info, on=None, how='right')
full_data

,gas/electric,rent length,bedrooms,price,bathrooms,master,storage?,balcony/garden,electric/solar,AC,...,furnished?,pets,smoke,livingroom,accessible?,where,city,phone number,name,cluster
0,3,6.0,4,3000.0,2.0,3,3,1,3,5,...,5,1,1,3,1,0,1.0,1000000,person0,94
1,3,12.0,4,3000.0,1.0,3,2,3,1,3,...,3,1,1,2,2,0,1.0,1000001,person1,76
2,3,12.0,6,8000.0,2.0,2,2,5,3,5,...,4,5,4,4,4,1,1.0,1000002,person2,10
3,4,12.0,3,4000.0,1.0,3,3,3,5,2,...,3,1,1,4,3,1,1.0,1000003,person3,23
4,1,24.0,3,4000.0,1.0,5,3,5,3,1,...,3,5,5,4,4,2,0.0,1000004,person4,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,5,12.0,5,2000.0,2.0,1,3,3,5,4,...,4,3,1,3,4,2,1.0,1001192,person1192,52
1193,4,12.0,3,3000.0,1.0,1,1,5,4,5,...,3,1,4,4,2,3,1.0,1001193,person1193,17
1194,5,12.0,2,3000.0,1.0,4,1,5,5,5,...,5,5,5,3,3,2,1.0,1001194,person1194,39
1195,5,12.0,4,2000.0,2.0,3,1,5,4,5,...,5,1,3,5,3,2,1.0,1001195,person1195,69


In [25]:
row=data.iloc[1]
row=pd.DataFrame([row])
kmeans_data.predict(row)

array([76])

In [26]:
match=full_data.loc[full_data["cluster"]==kmeans_data.predict(row)[0]]
match

,gas/electric,rent length,bedrooms,price,bathrooms,master,storage?,balcony/garden,electric/solar,AC,...,furnished?,pets,smoke,livingroom,accessible?,where,city,phone number,name,cluster
1,3,12.0,4,3000.0,1.0,3,2,3,1,3,...,3,1,1,2,2,0,1.0,1000001,person1,76
29,3,12.0,4,3000.0,1.5,1,1,3,3,3,...,3,1,2,2,1,0,1.0,1000029,person29,76
50,1,12.0,5,3000.0,2.0,3,3,2,3,3,...,4,2,1,4,2,0,1.0,1000050,person50,76
74,3,12.0,3,3000.0,1.5,1,3,3,4,3,...,3,3,2,3,3,0,1.0,1000074,person74,76
116,3,12.0,2,3000.0,1.0,1,3,1,1,1,...,3,1,1,2,3,2,1.0,1000116,person116,76
275,2,12.0,3,3000.0,1.5,3,2,2,3,1,...,3,1,1,3,4,0,1.0,1000275,person275,76
401,5,12.0,4,3000.0,1.5,1,3,1,4,4,...,1,1,1,4,3,2,1.0,1000401,person401,76
600,3,12.0,4,3000.0,1.0,3,2,3,1,3,...,3,1,1,2,2,0,1.0,1000600,person600,76
628,3,12.0,4,3000.0,1.5,3,1,3,3,3,...,3,1,2,2,1,0,1.0,1000628,person628,76
649,1,12.0,5,3000.0,2.0,4,3,2,3,3,...,4,2,1,4,2,0,1.0,1000649,person649,76


# FULL BUILDING

In [27]:
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9322 entries, 0 to 9321
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gas/electric       9322 non-null   int64  
 1   livingroom         9322 non-null   int64  
 2   bedrooms           9322 non-null   int64  
 3   bathrooms          9322 non-null   float64
 4   city               9322 non-null   int64  
 5   where              9322 non-null   object 
 6   languages          9322 non-null   object 
 7   house/building     9322 non-null   int64  
 8   storage?           9322 non-null   int64  
 9   furnished?         9322 non-null   int64  
 10  AC                 9322 non-null   int64  
 11  accessible?        9322 non-null   int64  
 12  Elevator           9322 non-null   int64  
 13  parking            9322 non-null   int64  
 14  pets               9322 non-null   int64  
 15  smoke              9322 non-null   int64  
 16  price              9322 

In [28]:
building = building.dropna(axis=0, how='any')

In [29]:
# {'Ariel': 0, 'Haifa': 1, 'Otef Azza': 2, 'Yehuda And Shomron': 3, 'South': 4, 'North': 5, 'Merkaz': 6, 'Tel Aviv': 7, 'Jerusalem': 8, 'Beer Sheba': 9}
u = building['where'].unique()
u_int = {}
for i in range(len(u)):
    u_int[u[i]] = i
w = []    
for index, row in building.iterrows():
    row['where'] = u_int.get(row['where'])
    w.append(row['where'])
building['where'] = w
building.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9322 entries, 0 to 9321
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gas/electric       9322 non-null   int64  
 1   livingroom         9322 non-null   int64  
 2   bedrooms           9322 non-null   int64  
 3   bathrooms          9322 non-null   float64
 4   city               9322 non-null   int64  
 5   where              9322 non-null   int64  
 6   languages          9322 non-null   object 
 7   house/building     9322 non-null   int64  
 8   storage?           9322 non-null   int64  
 9   furnished?         9322 non-null   int64  
 10  AC                 9322 non-null   int64  
 11  accessible?        9322 non-null   int64  
 12  Elevator           9322 non-null   int64  
 13  parking            9322 non-null   int64  
 14  pets               9322 non-null   int64  
 15  smoke              9322 non-null   int64  
 16  price              9322 

In [30]:
# {'EN': 0, 'AM': 1, 'FR': 2, 'SP': 3, '0': 4, 'RU': 5, 'AR': 6}
u = building['languages'].unique()
u_int = {}
for i in range(len(u)):
    u_int[u[i]] = i
w = []    
for index, row in building.iterrows():
    row['languages'] = u_int.get(row['languages'])
    w.append(row['languages'])
building['languages'] = w
building.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9322 entries, 0 to 9321
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gas/electric       9322 non-null   int64  
 1   livingroom         9322 non-null   int64  
 2   bedrooms           9322 non-null   int64  
 3   bathrooms          9322 non-null   float64
 4   city               9322 non-null   int64  
 5   where              9322 non-null   int64  
 6   languages          9322 non-null   int64  
 7   house/building     9322 non-null   int64  
 8   storage?           9322 non-null   int64  
 9   furnished?         9322 non-null   int64  
 10  AC                 9322 non-null   int64  
 11  accessible?        9322 non-null   int64  
 12  Elevator           9322 non-null   int64  
 13  parking            9322 non-null   int64  
 14  pets               9322 non-null   int64  
 15  smoke              9322 non-null   int64  
 16  price              9322 

In [31]:
personal_info_b = building[['phone number', 'name']]

building.drop('phone number', inplace=True, axis=1)
building.drop('name', inplace=True, axis=1)
building

,gas/electric,livingroom,bedrooms,bathrooms,city,where,languages,house/building,storage?,furnished?,...,accessible?,Elevator,parking,pets,smoke,price,rent length,public transpo,near construction,loud neighborhood
0,1,1,2,1.0,1,0,0,0,1,1,...,0,0,1,0,0,3000,1,1,0,1
1,0,1,2,1.0,1,0,1,1,0,1,...,1,1,0,0,0,3000,1,1,0,1
2,1,1,2,1.0,1,0,2,1,1,1,...,1,1,0,0,0,3000,1,1,0,1
3,0,1,1,1.0,1,1,0,1,1,1,...,0,1,1,1,0,2000,1,1,0,1
4,0,1,1,1.5,1,0,1,1,1,1,...,1,1,0,0,0,2200,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9317,1,0,1,1.0,1,0,4,1,1,1,...,1,1,0,0,0,2000,12,1,0,0
9318,0,1,2,1.0,1,0,0,1,0,1,...,0,1,0,1,0,3000,12,1,0,0
9319,0,0,1,1.0,1,0,4,1,1,1,...,0,0,0,0,0,1500,12,1,0,0
9320,0,0,1,1.0,1,0,4,1,1,1,...,0,0,0,0,0,1500,12,1,0,0


In [32]:
building.columns

Index(['gas/electric', 'livingroom', 'bedrooms', 'bathrooms', 'city', 'where',
       'languages', 'house/building', 'storage?', 'furnished?', 'AC',
       'accessible?', 'Elevator', 'parking', 'pets', 'smoke', 'price',
       'rent length', 'public transpo', 'near construction',
       'loud neighborhood'],
      dtype='object')

In [33]:
# Fitting K-Means to the dataset
kmeans_building = KMeans(n_clusters = 200, init = 'k-means++', random_state = 42)
y_kmeans_building = kmeans_building.fit_predict(building)
y_kmeans_building

array([158, 158, 158, ...,  74,  74, 159])

In [34]:
# # Visualising the clusters
# plt.figure(figsize=(15,7))
# t = building.values
# sns.scatterplot(t[y_kmeans_building == 0, 0], t[y_kmeans_building == 0, 1], color = 'yellow', label = 'Cluster 1',s=50)
# sns.scatterplot(t[y_kmeans_building == 1, 0], t[y_kmeans_building == 1, 1], color = 'blue', label = 'Cluster 2',s=50)
# sns.scatterplot(t[y_kmeans_building == 2, 0], t[y_kmeans_building == 2, 1], color = 'green', label = 'Cluster 3',s=50)
# sns.scatterplot(t[y_kmeans_building == 3, 0], t[y_kmeans_building == 3, 1], color = 'grey', label = 'Cluster 4',s=50)
# sns.scatterplot(t[y_kmeans_building == 4, 0], t[y_kmeans_building == 4, 1], color = 'orange', label = 'Cluster 5',s=50)
# sns.scatterplot(kmeans_building.cluster_centers_[:, 0], kmeans_building.cluster_centers_[:, 1], color = 'red', 
#                 label = 'Centroids',s=300,marker=',')
# plt.grid(False)
# plt.title('Clusters of customers')
# plt.xlabel('Annual Income (k$)')
# plt.ylabel('Spending Score (1-100)')
# plt.legend()
# plt.show()

In [35]:
# cl_building = AgglomerativeClustering(n_clusters=21, affinity='euclidean', linkage='ward')  
# cl_building.fit_predict(building)
# cl_building.labels_

In [36]:
personal_info_b['cluster'] = kmeans_building.labels_
personal_info_b

,phone number,name,cluster
0,110000,owner0,158
1,110001,owner1,158
2,110002,owner2,158
3,110003,owner3,118
4,110004,owner4,18
...,...,...,...
9317,119317,owner9317,35
9318,119318,owner9318,31
9319,119319,owner9319,74
9320,119320,owner9320,74


In [37]:
full_building = building.join(personal_info_b, on=None, how='right')
full_building

,gas/electric,livingroom,bedrooms,bathrooms,city,where,languages,house/building,storage?,furnished?,...,pets,smoke,price,rent length,public transpo,near construction,loud neighborhood,phone number,name,cluster
0,1,1,2,1.0,1,0,0,0,1,1,...,0,0,3000,1,1,0,1,110000,owner0,158
1,0,1,2,1.0,1,0,1,1,0,1,...,0,0,3000,1,1,0,1,110001,owner1,158
2,1,1,2,1.0,1,0,2,1,1,1,...,0,0,3000,1,1,0,1,110002,owner2,158
3,0,1,1,1.0,1,1,0,1,1,1,...,1,0,2000,1,1,0,1,110003,owner3,118
4,0,1,1,1.5,1,0,1,1,1,1,...,0,0,2200,1,1,0,1,110004,owner4,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9317,1,0,1,1.0,1,0,4,1,1,1,...,0,0,2000,12,1,0,0,119317,owner9317,35
9318,0,1,2,1.0,1,0,0,1,0,1,...,1,0,3000,12,1,0,0,119318,owner9318,31
9319,0,0,1,1.0,1,0,4,1,1,1,...,0,0,1500,12,1,0,0,119319,owner9319,74
9320,0,0,1,1.0,1,0,4,1,1,1,...,0,0,1500,12,1,0,0,119320,owner9320,74


In [38]:
groups = full_building.groupby("cluster")["name"].count()
groups

cluster
0      12
1      68
2      33
3      19
4      74
       ..
195    20
196    38
197    55
198    31
199    22
Name: name, Length: 200, dtype: int64

# NEW INSTANCE

In [39]:
row = building.iloc[0]
print(row)
row=pd.DataFrame([row])

row.head()

gas/electric            1.0
livingroom              1.0
bedrooms                2.0
bathrooms               1.0
city                    1.0
where                   0.0
languages               0.0
house/building          0.0
storage?                1.0
furnished?              1.0
AC                      1.0
accessible?             0.0
Elevator                0.0
parking                 1.0
pets                    0.0
smoke                   0.0
price                3000.0
rent length             1.0
public transpo          1.0
near construction       0.0
loud neighborhood       1.0
Name: 0, dtype: float64


,gas/electric,livingroom,bedrooms,bathrooms,city,where,languages,house/building,storage?,furnished?,...,accessible?,Elevator,parking,pets,smoke,price,rent length,public transpo,near construction,loud neighborhood
0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,3000.0,1.0,1.0,0.0,1.0


In [40]:
kmeans_building.predict(row)

array([158])